In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import os
import sys
import time
import logging
import re
from commons import *
from gan_arch import *
from datetime import datetime

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
else:
    print("No compatible GPUs found")


In [ ]:
METU_RAW_PATH = '/qarr/studia/magister/datasets/METU/930k_logo_v3/'
METU_DATASET_PATH = '/home/zenfur/magister/resized_930k_logo/'
EVAL_ORIGIN_PATH = '/qarr/studia/magister/datasets/METU/query_reversed/'
EVAL_DATASET_PATH = '/home/zenfur/magister/metu_eval_256sq/'
LOG_DIR = "siamese_logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
MODEL_SAVE_NAME = "siamese_model_1"
TESTING=False

# Preparing the dataset pipeline and testing

In [ ]:
imagesList = tf.io.matching_files(EVAL_DATASET_PATH + "*.jpg")

@tf.function
def tf_get_filename(path):
    return tf.strings.regex_replace(path, "[^/]*/", "")


#@tf.function
def tf_read_image(path):
    # Retrieving the group number from file name
    img = tf.io.read_file(path)
    return tf.image.decode_jpeg(img, channels=3, dct_method='INTEGER_ACCURATE')


def tf_get_class_from_name(path):
    filename = tf_get_filename(path)
    group_number = tf.strings.to_number(
        tf.strings.regex_replace(filename, "-.*$", ""), 
        out_type=tf.dtypes.int32
    )
    return group_number

#@tf.function
def tf_convert_and_normalize_img(img):
    c = tf.constant(256.0, dtype=tf.dtypes.float32)
    img = tf.cast(img, tf.dtypes.float32)
    return tf.math.divide(img, c)


evalpathsDB = tf.data.Dataset.from_tensor_slices(imagesList)

DBlen = len(imagesList)

evalDB = (      evalpathsDB.map(tf_read_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                 .batch(32)
                 .map(tf_convert_and_normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)

evalGrps = (      evalpathsDB.map(tf_get_class_from_name, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                   .batch(32)
)


# Counting and preparing images into groups by name
By convention, discarding images from group 0, as they have been manually inserted as the examples that differ from the rest sampled from 930k METU dataset.

In [ ]:
labs = list(evalGrps.unbatch().as_numpy_iterator())

In [ ]:
imageGroupsDct = dict()
for i, l in enumerate(labs):
    last_count = imageGroupsDct.get(l,(i,0))
    imageGroupsDct[l] = (last_count[0], last_count[1] + 1)
del(imageGroupsDct[0])

In [ ]:
imageGroups = list(imageGroupsDct.items())
avoids, starts, lengths, seeds = [], [], [], []

# Generating the random seeds for custom random sequence generators that iterate over triplets from each group
# For the sake of being repeatable, fixing seed
np.random.seed(949127843)
for igrp in imageGroups:
    for i in range(igrp[1][1]):
        avoids.append(i+igrp[1][0])
        starts.append(igrp[1][0])
        lengths.append(igrp[1][1])
        seeds.append(np.random.randint(0, high=10000000))
        

### Custom triples sampler based on linear congruent generator 
The generator has period equal NumberOfUnlikeSamples*(NumberOfAlikeSamples-1)

In [ ]:
def triplet_generator(start, length, avoid, n, initial, prime=756212269):
    """
    results in sequence of triplets (<start:start+length>, <avoid>, <0:n-1 excluding start:start+length-1>)
    """
    start = np.array(start, dtype=np.int32)
    length = np.array(length, dtype=np.int32)
    avoid = np.array(avoid, dtype=np.int32)
    initial = np.array(initial, dtype=np.int32)
    current = initial
    unlikeCount = (n-length)
    length = length - 1
    modulo_base = length * unlikeCount
    multiplier = modulo_base*11*3+1
    while True:
        current = (multiplier * current + prime) % modulo_base
        like = (current % length) + start
        unlike = current // length
        like = like + (like >= avoid)
        unlike = unlike + ((unlike >= start) * (length+1))
        like = np.expand_dims(like, axis=1)
        unlike = np.expand_dims(unlike, axis=1)
        for triplet in np.concatenate((like, np.expand_dims(avoid, axis=1), unlike), axis=1):
            yield triplet

In [ ]:
gen = triplet_generator(starts, lengths, avoids, len(labs), seeds)

Testing the generator 

In [ ]:
if TESTING:
    # Test: group 1:3, n=6, x = 2
    g = triplet_generator([1], [3], [2], 6, [423432231])
    s = set()
    for i in range(2*3):
        t = next(g)
        print(t)
        if tuple(t) in s:
            break
        else:
            s.add(tuple(t))
    assert i == (2*3)-1

In [ ]:
if TESTING:
    for i in range(10):
        print(next(gen))

## Building the pipeline segments 

Converting the database of samples into numpy array, since it can fit into RAM memory to save time.

In [ ]:
imagesTable = tf_db_to_array(evalDB, DBlen)

In [ ]:
def get_triplet_by_index(triplet):
    # @triplet: tuple/tensor of indices in the images table
    return tf.gather(imagesTable, triplet)

Initialising the common source of random numbers

In [ ]:
rng = tf.random.Generator.from_seed(41431)

Defining the data augmentation functions for the pipeline.

In [ ]:
def random_slice_224x224(image, seed):
    return tf.image.stateless_random_crop(image, [224,224,3], seed)


def random_rotate(image, seed):
    if seed > 3:
        seed = 0
    return tf.image.rot90(image, k=seed)
    
    
def augment(image):
    seeds = [rng.make_seeds(2)[0], rng.uniform([], minval=0, maxval=5, dtype=tf.int32)]
    sditer = iter(seeds)
    
    image = random_slice_224x224(image, next(sditer))
    
    image = random_rotate(image, next(sditer))
    
    return image

# @tf.function
# def random_slice_224x224(img, seed):
#     # TODO: get zipped random input to make it deterministic and reproducable
#     # assuming 256x256x3 size
#     randomCrop = tf.concat([tf.random.uniform((2,), minval=0, maxval=256-224, dtype=tf.dtypes.int32), 
#                            tf.constant([0])], axis=0)
#     return tf.slice(img, randomCrop, [224,224,3])

### The input data pipeline

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 3
df = (tf.data.Dataset.from_generator(triplet_generator,
                                    args = [starts, lengths, avoids, len(labs), seeds],
                                    output_signature=tf.TensorSpec(shape=(3), dtype=tf.int32))
      .shuffle(256)
      .map(get_triplet_by_index, num_parallel_calls=AUTOTUNE)
      .unbatch()
      .map(augment, num_parallel_calls=AUTOTUNE)#, deterministic=True) # transforms of data augmentation, should be deterministic...
      .batch(3*BATCH_SIZE)
      .prefetch(2)#AUTOTUNE)
)

Sampling and testing the pipeline

In [ ]:
if TESTING:
    img = get_triplet_by_index(tf.constant(next(triplet_generator(starts, lengths, avoids, len(labs), seeds))))
    f, subplots = plt.subplots(1,3)
    for i, sb in enumerate(subplots):
        sb.imshow(img[i])
        


In [ ]:
if TESTING or True:
    img = df.take(1)
    img = next(iter(img))
    f, subplots = plt.subplots(1,3)
    for i, sb in enumerate(subplots):
        sb.imshow(img[i])

## Siamese model

Importing the  pre-trained VGG16 model with weights from imagenet without classification part.

Adding 2 dense layers on top of convolutions for 4096 representation.

In [ ]:
vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
#     input_shape=None,
#     pooling=None,
)

# TODO: load model if saved is found
siamese_base = tf.keras.models.Sequential()
for layer in (vgg16,
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(4096, activation='relu'),
                tf.keras.layers.Dense(4096, activation='relu')
             ):
    siamese_base.add(layer)

In [ ]:
if TESTING:
    tf.keras.utils.plot_model(vgg16, "vgg16.png", show_shapes=True)

In [ ]:
# batch=32
# input_alike = tf.keras.layers.Input(shape=(224,224,3), batch_size=batch, name="alike_anchor")
# input_anchor = tf.keras.layers.Input(shape=(224,224,3), batch_size=batch, name="anchor")
# input_unlike = tf.keras.layers.Input(shape=(224,224,3), batch_size=batch, name="unlike_anchor")

# alike_triplet = siamese_base(input_alike)
# anchor_triplet = siamese_base(input_anchor)
# unlike_triplet = siamese_base(input_unlike)

# # triplet_siamese = tf.keras.models.Model(inputs=[input_alike, input_anchor, input_unlike], 
# #                                         outputs=[alike_triplet, anchor_triplet, unlike_triplet])

### Siamese triplet loss function 

In [ ]:
def triplet_loss(alike, anchor, unlike, margin=1.0):
#     together = tf.concat([alike, anchor, unlike], axis=0)
#     maxims = tf.math.reduce_max(together, axis=0)
#     minims = tf.math.reduce_min(together, axis=0)
#     difference = maxims-minims
#     alike = (alike - minims)/difference
#     anchor = (anchor - minims)/difference
#     unlike = (unlike - minims)/difference
    a = tf.norm(alike-anchor)
    b = tf.norm(unlike-anchor)
    return tf.maximum(a + margin - b, 0.0)

## Defining custom model

In [ ]:
class TripletSiamese(tf.keras.models.Model):
    def __init__(self, shared_net):
        super(TripletSiamese, self).__init__()
        self.siamese_base = shared_net

    def compile(self, optimizer, loss_margin):
        super(TripletSiamese, self).compile()
        self.optimizer = optimizer
        self.loss = triplet_loss#lambda a,b,c: triplet_loss(a,b,c, margin=loss_margin)

    def train_step(self, input_triplets):
        
        with tf.GradientTape(persistent=True) as tape:
            # training and calculating the error function gradient
            representations = self.siamese_base(input_triplets, training=True)
            #representations = tf.keras.utils.normalize(representations, axis=1)
            alike = tf.strided_slice(representations, [0,0], tf.shape(representations), strides=[3,1])
            anchor = tf.strided_slice(representations, [1,0], tf.shape(representations), strides=[3,1])
            unlike = tf.strided_slice(representations, [2,0], tf.shape(representations), strides=[3,1])
            loss = self.loss(alike, anchor, unlike)
        grads = tape.gradient(loss, self.siamese_base.trainable_weights)

#         with tf.GradientTape() as tape:
#             alike = self.siamese_base(input_triplets[0], training=True)
#         grads1 = tape.gradient(alike, self.siamese_base.trainable_weights) 
        
#         with tf.GradientTape() as tape:
#             anchor = self.siamese_base(input_triplets[1], training=True)
#         grads2 = tape.gradient(anchor, self.siamese_base.trainable_weights) 
        
#         with tf.GradientTape() as tape:
#             unlike = self.siamese_base(input_triplets[2], training=True)
#         grads3 = tape.gradient(unlike, self.siamese_base.trainable_weights) 
        
#         grads1 *= 2*(alike - anchor)
#         grads2 *= 2*(unlike - alike)
#         grads3 *= 2*(alike - unlike)
        
#         grads = np.mean([grads1, grads2, grads3])# ...
            
        self.optimizer.apply_gradients(zip(grads, self.siamese_base.trainable_weights))

        return {"loss": loss}

## Initialising and compiling the model

In [ ]:
triplet_model = TripletSiamese(siamese_base)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
triplet_model.compile(opt, triplet_loss)

In [ ]:
# TODO: add saving/loading model

# Calling the training with hooks

In [ ]:
N = np.sum([a[1][1] for a in imageGroupsDct.items()])

In [ ]:
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = LOG_DIR)#, histogram_freq=4)

In [ ]:
# Epochs to train
train_for = 200
try:
    triplet_model.fit(df, 
                  initial_epoch=0,
                  epochs=3,#train_for, 
                  steps_per_epoch=N//BATCH_SIZE,
                  callbacks=[tboard_callback]#, callbackCheckpoint]
                 ) # batch_size unspecified since it's generated by generator
except KeyboardInterrupt as e:
    print("Interrupted")
    
if MODEL_SAVE_NAME is not None:
    triplet_model.save(MODEL_SAVE_NAME)

In [ ]:
    
if MODEL_SAVE_NAME is not None:
    triplet_model.save(MODEL_SAVE_NAME)